In [1]:
!pip install datasets transformers torch torchvision faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install torch torchvision pillow sentence-transformers ftfy git+https://github.com/openai/CLIP.git pandas numpy

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-mtpgx5jm
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-mtpgx5jm
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.3 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=6cefdd5ae81cbc991267f97e31bee41bb91f44b7dd3c1835e01da574d32c9e92
  Stored in directory: /tmp/pip-ephem-wheel-cache-lf7clqfs/wheels/3f/7c/a4/9b490845988bf7a4db33674d52f709f088f64392063872eb9a
Successfully built clip


In [3]:
import pandas as pd
import numpy as np
import torch
import faiss
from datasets import load_dataset
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import requests
from io import BytesIO
from datetime import datetime
from tqdm import tqdm
import torch
import clip
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
print("Loaded Model")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

Loaded Model


# APPLY CLIP Model on the Beauty Products Dataset

In [5]:
# Load the dataset directly from GitHub
url = 'https://raw.githubusercontent.com/tanalam565/Article-Categorization/main/Amazon_beauty_products_queries.csv'
df_query = pd.read_csv(url)

# Display the first few rows to confirm
df_query.head()

,Queries,Product_link,Product_title,Image_link,Product_description
0,Nose Rings Bone Stud,https://www.amazon.com/Hoeudjo-Surgical-Zircon...,Hoeudjo Nose Studs Surgical Steel Cubic Zircon...,https://m.media-amazon.com/images/I/618A3KC3Ua...,"MaterialMetal, Rhinestone Metal typeStainless ..."
1,Nose Rings Bone Stud,https://www.amazon.com/Zolure-Surgical-20-30PC...,Zolure 18G 20G 22G Nose Pin Bone Screws Studs ...,https://m.media-amazon.com/images/I/71HH1cIQNe...,"MaterialStainless Steel,Steel,Stone,Diamond Me..."
2,Nose Rings Bone Stud,https://www.amazon.com/Surgical-Stainless-Hypo...,"Cute Nose Rings for Women, Surgical Stainless ...",https://m.media-amazon.com/images/I/51pfB-1xZ8...,MaterialStainless Steel Metal typeStainless St...
3,Nose Rings Bone Stud,https://www.amazon.com/Ocptiy-Surgical-Stainle...,Ocptiy 20G 18G 16G Push in Theadless Nose Ring...,https://m.media-amazon.com/images/I/614DppQpTc...,"MaterialStainless Steel, Cubic Zirconia Metal ..."
4,Nose Rings Bone Stud,https://www.amazon.com/Thunaraz-60pcs-Stainles...,THUNARAZ Straight L Shaped Stainless Steel Nos...,https://m.media-amazon.com/images/I/510aIu8nTZ...,"MaterialStainless Steel,Crystal Metal typeStai..."


In [6]:
# Shape of the dataset
df_query.shape

(250, 5)

In [7]:
# Check the null values
df_query.isnull().sum()

,0
Queries,0
Product_link,0
Product_title,0
Image_link,0
Product_description,0


In [8]:
# Print the query column
df_query['Queries'].value_counts()

,count
Queries,
Nose Rings Bone Stud,5
Love Lipstick,5
Nail Files,5
"Shower Gel, Body Lotion, and Body Splash",5
Scrunchy Updo Wavy Straight Hair Bun,5
Mousse Relaxed Hair,5
10A Straight Human Hair Bundles with Closure,5
Blitzby Tattoo Lipstick Cigarette Cotton Swab,5
Moisturizing Facial Mask,5


# APPLY CLIP MODEL

### Define the embeddings for text and image

In [9]:
def get_text_embedding(text):
    # Truncate the text to the maximum context length
    text = text[:77]
    # Use clip.tokenize
    text_input = clip.tokenize([text]).to(device)
    with torch.no_grad():
        # Use clip_model.get_text_features()
        text_embedding = clip_model.get_text_features(text_input)
    return text_embedding.cpu().numpy().flatten()

def get_image_embedding(image_url):
    try:
        response = requests.get(image_url)
        image = Image.open(BytesIO(response.content)).convert("RGB")
        # Use clip_processor instead of preprocess
        image_input = clip_processor(images=image, return_tensors="pt").to(device)
        with torch.no_grad():
            # Use clip_model.get_image_features()
            image_embedding = clip_model.get_image_features(image_input.pixel_values)
        return image_embedding.cpu().numpy().flatten()
    except:
        return np.zeros(512)  # Fallback if image fails to load

In [10]:
# Initialize columns for embeddings
df_query["text_embedding"] = None
df_query["image_embedding"] = None
df_query["multimodal_embedding"] = None

# Compute embeddings for each row
for idx, row in tqdm(df_query.iterrows(), total=len(df_query)):
    # Text embedding (query + product title/description)
    combined_text = f"{row['Queries']} | {row['Product_title']} | {row['Product_description']}"
    text_emb = get_text_embedding(combined_text)

    # Image embedding
    image_emb = get_image_embedding(row["Image_link"])

    # Combine embeddings (normalized concatenation)
    multimodal_emb = np.concatenate([text_emb, image_emb])
    multimodal_emb = multimodal_emb / np.linalg.norm(multimodal_emb)  # Normalize

    df_query.at[idx, "text_embedding"] = text_emb.tolist()
    df_query.at[idx, "image_embedding"] = image_emb.tolist()
    df_query.at[idx, "multimodal_embedding"] = multimodal_emb.tolist()

100%|██████████| 250/250 [00:26<00:00,  9.26it/s]


In [11]:
results = []
queries = df_query["Queries"].unique()

for query in queries:
    query_text_emb = get_text_embedding(query)
    query_multimodal_emb = np.concatenate([query_text_emb, np.zeros(512)])  # Placeholder for image
    query_multimodal_emb = query_multimodal_emb / np.linalg.norm(query_multimodal_emb)

    query_products = df_query[df_query["Queries"] == query]
    product_embeddings = np.stack(query_products["multimodal_embedding"].values)

    similarities = cosine_similarity(
        query_multimodal_emb.reshape(1, -1),
        product_embeddings
    )[0]

    for idx, row in query_products.iterrows():
        results.append({
            "Query": query,
            "Product_Title": row["Product_title"],
            "Product_Description": row["Product_description"],
            "Image_Link": row["Image_link"],
            "Similarity_Score": similarities[idx - query_products.index[0]]
        })

results_df = pd.DataFrame(results)

In [12]:
# Prints the first 5 rows
results_df.head()

,Query,Product_Title,Product_Description,Image_Link,Similarity_Score
0,Nose Rings Bone Stud,Hoeudjo Nose Studs Surgical Steel Cubic Zircon...,"MaterialMetal, Rhinestone Metal typeStainless ...",https://m.media-amazon.com/images/I/618A3KC3Ua...,0.468211
1,Nose Rings Bone Stud,Zolure 18G 20G 22G Nose Pin Bone Screws Studs ...,"MaterialStainless Steel,Steel,Stone,Diamond Me...",https://m.media-amazon.com/images/I/71HH1cIQNe...,0.417197
2,Nose Rings Bone Stud,"Cute Nose Rings for Women, Surgical Stainless ...",MaterialStainless Steel Metal typeStainless St...,https://m.media-amazon.com/images/I/51pfB-1xZ8...,0.477182
3,Nose Rings Bone Stud,Ocptiy 20G 18G 16G Push in Theadless Nose Ring...,"MaterialStainless Steel, Cubic Zirconia Metal ...",https://m.media-amazon.com/images/I/614DppQpTc...,0.484552
4,Nose Rings Bone Stud,THUNARAZ Straight L Shaped Stainless Steel Nos...,"MaterialStainless Steel,Crystal Metal typeStai...",https://m.media-amazon.com/images/I/510aIu8nTZ...,0.512862


In [13]:
import altair as alt

# Group by query and calculate the average similarity score
average_similarity = results_df.groupby('Query')['Similarity_Score'].mean().reset_index()

# Create the Altair chart
chart = alt.Chart(average_similarity).mark_bar().encode(
    x='Query',
    y='Similarity_Score'
).properties(
    title='Average Similarity Scores for Each Query',
    width=600,
    height=400
)

chart


alt.Chart(...)

In [14]:
# Calculate average similarity
average_similarity = results_df["Similarity_Score"].mean()
print(f"Average Similarity Score: {average_similarity:.4f}")

Average Similarity Score: 0.5086


# ZERO SHOT CLASSIFICATION TASK

In [15]:
# Load model and preprocess
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

100%|███████████████████████████████████████| 338M/338M [00:19<00:00, 18.5MiB/s]


In [16]:
# Function to define zero shot classification
def zero_shot_classify(image_url, class_labels, query=None):

    try:
        # Load image
        response = requests.get(image_url, timeout=5)
        image = Image.open(BytesIO(response.content)).convert("RGB")
        image_input = preprocess(image).unsqueeze(0).to(device)

        # Prepare text inputs (combine query with class labels)
        if query:
            text_inputs = torch.cat([clip.tokenize(f"{query}: {label}") for label in class_labels]).to(device)
        else:
            text_inputs = torch.cat([clip.tokenize(label) for label in class_labels]).to(device)

        # Calculate features
        with torch.no_grad():
            image_features = model.encode_image(image_input)
            text_features = model.encode_text(text_inputs)

        # Compute similarity and softmax
        similarity = (image_features @ text_features.T).softmax(dim=-1)
        probs = similarity.cpu().numpy().flatten()

        return {label: float(prob) for label, prob in zip(class_labels, probs)}

    except Exception as e:
        print(f"Error processing {image_url}: {e}")
        return {label: 0.0 for label in class_labels}

In [17]:
class_labels = [
    "matte lipstick",
    "glossy lipstick",
    "metallic lipstick",
    "long-lasting lipstick",
    "vegan lipstick"
]

# Example: Classify an image
image_url = "https://m.media-amazon.com/images/I/61ymji-L-uL._SX522_.jpg"
results = zero_shot_classify(image_url, class_labels, query="Love Lipstick")
print(results)

{'matte lipstick': 0.2158203125, 'glossy lipstick': 0.04815673828125, 'metallic lipstick': 0.04742431640625, 'long-lasting lipstick': 0.517578125, 'vegan lipstick': 0.1707763671875}


In [18]:
# Classify all products for a query
def classify_products(df, query, class_labels): # Added df as an argument
    df_class = df_query[df_query["Queries"] == query]
    results = []
    for _, row in df_class.iterrows():
        probs = zero_shot_classify(row["Image_link"], class_labels, query)
        results.append({
            "Product_Title": row["Product_title"],
            "Class_Probabilities": probs,
            "Top_Class": max(probs, key=probs.get)
        })
    return pd.DataFrame(results)

# Usage
lipstick_labels = ["matte", "glossy", "metallic", "long-lasting", "vegan"]
classify_products(df_query, "Love Lipstick", lipstick_labels) # Passed df_query to the function

,Product_Title,Class_Probabilities,Top_Class
0,Clinique Clinique Pop Longwear Lipstick | Long...,"{'matte': 0.331787109375, 'glossy': 0.02258300...",long-lasting
1,"Glitter Lipstick, Metallic Lipstick For Women,...","{'matte': 0.232421875, 'glossy': 0.08288574218...",long-lasting
2,Lancôme Lancôme L'Absolu Rouge Hydrating Cream...,"{'matte': 0.6455078125, 'glossy': 0.0142593383...",matte
3,Clinique Clinique Dramatically Different Lipst...,"{'matte': 0.382080078125, 'glossy': 0.01901245...",long-lasting
4,"Revlon Super Lustrous Lipstick, Creamy Formula...","{'matte': 0.4990234375, 'glossy': 0.1113891601...",matte


# ZERO SHOT WITH COSINE SIMILARITY

In [19]:
def zero_shot_cosine_similarity(image_url, class_labels, query=None):

    try:
        # Load and preprocess image
        response = requests.get(image_url, timeout=5)
        image = Image.open(BytesIO(response.content)).convert("RGB")
        image_input = preprocess(image).unsqueeze(0).to(device)

        # Prepare text inputs (combine query with labels if provided)
        if query:
            texts = [f"{query}: {label}" for label in class_labels]
        else:
            texts = class_labels
        text_inputs = clip.tokenize(texts).to(device)

        # Get embeddings
        with torch.no_grad():
            image_embedding = model.encode_image(image_input).cpu().numpy()
            text_embeddings = model.encode_text(text_inputs).cpu().numpy()

        # Compute cosine similarity
        similarities = cosine_similarity(image_embedding, text_embeddings)[0]
        return {label: float(sim) for label, sim in zip(class_labels, similarities)}

    except Exception as e:
        print(f"Error processing {image_url}: {e}")
        return {label: 0.0 for label in class_labels}

In [20]:
# For "Love Lipstick" query
lipstick_labels = [
    "matte lipstick",
    "glossy lipstick",
    "metallic lipstick",
    "long-lasting lipstick"
]

# For "Nose Rings Bone Stud" query
nose_ring_labels = [
    "surgical steel nose ring",
    "bone stud nose piercing",
    "cubic zirconia nose stud"
]

In [21]:
# Example 1: Lipstick
image_url = "https://m.media-amazon.com/images/I/61ymji-L-uL._SX522_.jpg"
results = zero_shot_cosine_similarity(image_url, lipstick_labels, query="Love Lipstick")
print(results)

{'matte lipstick': 0.29239399342169065, 'glossy lipstick': 0.29431035339795053, 'metallic lipstick': 0.296679650637224, 'long-lasting lipstick': 0.30294843586585873}




# Calculate the text and image Image similarity separately and then find the combined cosine similarity between query and recommended products.


In [22]:
def get_query_vs_recommendations_similarity(query, product_title, product_description, image_url):
    """
    Compute cosine similarity between:
    - The search query (text)
    - Each recommended product (text + image)

    Returns:
        dict: {"text_similarity": float, "image_similarity": float, "combined_similarity": float}
    """
    try:
        # Text Embeddings
        with torch.no_grad():
            query_text_embed = model.encode_text(clip.tokenize([query]).to(device))
            product_text = f"{product_title} | {product_description}"
            product_text_embed = model.encode_text(clip.tokenize([product_text], truncate=True).to(device))

            # Image Embedding
            response = requests.get(image_url, timeout=5)
            image = Image.open(BytesIO(response.content)).convert("RGB")
            image_embed = model.encode_image(preprocess(image).unsqueeze(0).to(device))

        # Convert to numpy arrays
        query_np = query_text_embed.cpu().detach().numpy()
        product_text_np = product_text_embed.cpu().detach().numpy()
        image_np = image_embed.cpu().detach().numpy()

        # Compute Cosine Similarities
        text_sim = cosine_similarity(query_np, product_text_np)[0][0]
        image_sim = cosine_similarity(query_np, image_np)[0][0]

        # Combined similarity (average of text and image)
        combined_sim = (text_sim + image_sim) / 2

        return {
            "text_similarity": float(text_sim),
            "image_similarity": float(image_sim),
            "combined_similarity": float(combined_sim)
        }

    except Exception as e:
        print(f"Error processing {product_title}: {str(e)}")
        return {
            "text_similarity": 0.0,
            "image_similarity": 0.0,
            "combined_similarity": 0.0
        }

In [23]:
# Compute similarities for each query-product pair
results = []
for _, row in df_query.iterrows():
    sim_scores = get_query_vs_recommendations_similarity(
        query=row["Queries"],
        product_title=row["Product_title"],
        product_description=row["Product_description"],
        image_url=row["Image_link"]
    )
    results.append({
        "Query": row["Queries"],
        "Product_Title": row["Product_title"],
        "Text_Similarity": sim_scores["text_similarity"],
        "Image_Similarity": sim_scores["image_similarity"],
        "Combined_Similarity": sim_scores["combined_similarity"]
    })

# Convert to DataFrame
similarity_df = pd.DataFrame(results)

In [24]:
# Prints the first 5 rows of the dataframe
similarity_df.head()

,Query,Product_Title,Text_Similarity,Image_Similarity,Combined_Similarity
0,Nose Rings Bone Stud,Hoeudjo Nose Studs Surgical Steel Cubic Zircon...,0.659346,0.281321,0.470333
1,Nose Rings Bone Stud,Zolure 18G 20G 22G Nose Pin Bone Screws Studs ...,0.602719,0.265782,0.434251
2,Nose Rings Bone Stud,"Cute Nose Rings for Women, Surgical Stainless ...",0.737386,0.304826,0.521106
3,Nose Rings Bone Stud,Ocptiy 20G 18G 16G Push in Theadless Nose Ring...,0.624050,0.285959,0.455005
4,Nose Rings Bone Stud,THUNARAZ Straight L Shaped Stainless Steel Nos...,0.708585,0.325344,0.516965


In [25]:
# Calculate the mean of similarity scores
average_similarity = similarity_df["Combined_Similarity"].mean()
print(f"Average Similarity Score: {average_similarity:.4f}")

Average Similarity Score: 0.4610


## So, Better Average Similarity Score: 0.5086 (Multimodel Embedding Using CLIP Model)

# Apply CLIP MOdel in the Appliances dataset

In [26]:
# URL for the appliances dataset
url = 'https://raw.githubusercontent.com/tanalam565/Article-Categorization/main/Amazon_appliances_queries.csv'

# Load the CSV into a DataFrame
df_query = pd.read_csv(url)

# Display the first few rows
df_query.head()

,Queries,Product_link,Product_title,Image_link,Product_description
0,Replacement Filter Pack Compatible with Coway ...,https://www.amazon.com/Byseacore-AP-1216-FP-Re...,AP-1216-FP Replacement Filter Compatible with ...,https://m.media-amazon.com/images/I/817zez3TPb...,Brand\tByseacore Material\tPolypropylene Produ...
1,Replacement Filter Pack Compatible with Coway ...,https://www.amazon.com/PUREBURG-Complete-Repla...,PUREBURG AP-1216L Replacement True HEPA Filter...,https://m.media-amazon.com/images/I/71cOAMPFfk...,Brand\tPUREBURG Material\tTrue HEPA MERV Ratin...
2,Replacement Filter Pack Compatible with Coway ...,https://www.amazon.com/AP-1216-FP-Replacement-...,AP-1216-FP Replacement Filter for Coway AP-121...,https://m.media-amazon.com/images/I/71yzYQwGUG...,"Brand\tDAODEFUL Material\tTrue HEPA, Active Ca..."
3,Replacement Filter Pack Compatible with Coway ...,https://www.amazon.com/Byseacore-AP-1216-FP-Fi...,1216L Replacement Filter Compatible with Coway...,https://m.media-amazon.com/images/I/81tFK-tnbU...,Brand\tByseacore Material\tPolypropylene Produ...
4,Replacement Filter Pack Compatible with Coway ...,https://www.amazon.com/LifeSupplyUSA-Carbon-Re...,LifeSupplyUSA Complete Replacement Filter Set ...,https://m.media-amazon.com/images/I/71qcUBK5xp...,Brand\tLifeSupplyUSA Material\tFiberglass Prod...


In [27]:
# Shape of the dataset
df_query.shape

(250, 5)

In [28]:
# Check the null values
df_query.isnull().sum()

,0
Queries,0
Product_link,0
Product_title,0
Image_link,0
Product_description,0


In [29]:
# Print the query column
df_query['Queries'].value_counts()

,count
Queries,
Replacement Filter Pack Compatible with Coway AP-1216L Air Purifier,5
"Convection Motor Compatible With Samsung Range Product Name DG31-00005A, AP4338602, PS4240735",5
Frigidaire 5304461262 Dryer Maintenance Kit,5
"Teflon Replacement Hot Plate Cover - 8""W X 15""L",5
Frigidaire 316448700 Light Switch,5
Whirlpool Part Number 4318011,5
NRP 6-pack Activated Water Charcoal Filter,5
285748 Washer Agitator Repair Kit - Compatible Whirlpool Maytag Kenmore,5
XPWB2X9154 Round Style Oven or Broiler Igniter,5


# APPLY CLIP MODEL

### Define the embeddings for text and image

In [30]:
def get_text_embedding(text):
    # Truncate the text to the maximum context length
    text = text[:77]
    # Use clip.tokenize
    text_input = clip.tokenize([text]).to(device)
    with torch.no_grad():
        # Use clip_model.get_text_features()
        text_embedding = clip_model.get_text_features(text_input)
    return text_embedding.cpu().numpy().flatten()

def get_image_embedding(image_url):
    try:
        response = requests.get(image_url)
        image = Image.open(BytesIO(response.content)).convert("RGB")
        # Use clip_processor instead of preprocess
        image_input = clip_processor(images=image, return_tensors="pt").to(device)
        with torch.no_grad():
            # Use clip_model.get_image_features()
            image_embedding = clip_model.get_image_features(image_input.pixel_values)
        return image_embedding.cpu().numpy().flatten()
    except:
        return np.zeros(512)  # Fallback if image fails to load

In [31]:
# Initialize columns for embeddings
df_query["text_embedding"] = None
df_query["image_embedding"] = None
df_query["multimodal_embedding"] = None

# Compute embeddings for each row
for idx, row in tqdm(df_query.iterrows(), total=len(df_query)):
    # Text embedding (query + product title/description)
    combined_text = f"{row['Queries']} | {row['Product_title']} | {row['Product_description']}"
    text_emb = get_text_embedding(combined_text)

    # Image embedding
    image_emb = get_image_embedding(row["Image_link"])

    # Combine embeddings (normalized concatenation)
    multimodal_emb = np.concatenate([text_emb, image_emb])
    multimodal_emb = multimodal_emb / np.linalg.norm(multimodal_emb)  # Normalize

    df_query.at[idx, "text_embedding"] = text_emb.tolist()
    df_query.at[idx, "image_embedding"] = image_emb.tolist()
    df_query.at[idx, "multimodal_embedding"] = multimodal_emb.tolist()

100%|██████████| 250/250 [00:19<00:00, 12.83it/s]


In [32]:
results = []
queries = df_query["Queries"].unique()

for query in queries:
    query_text_emb = get_text_embedding(query)
    query_multimodal_emb = np.concatenate([query_text_emb, np.zeros(512)])  # Placeholder for image
    query_multimodal_emb = query_multimodal_emb / np.linalg.norm(query_multimodal_emb)

    query_products = df_query[df_query["Queries"] == query]
    product_embeddings = np.stack(query_products["multimodal_embedding"].values)

    similarities = cosine_similarity(
        query_multimodal_emb.reshape(1, -1),
        product_embeddings
    )[0]

    for idx, row in query_products.iterrows():
        results.append({
            "Query": query,
            "Product_Title": row["Product_title"],
            "Product_Description": row["Product_description"],
            "Image_Link": row["Image_link"],
            "Similarity_Score": similarities[idx - query_products.index[0]]
        })

results_df = pd.DataFrame(results)

In [33]:
# Pront the first 5 rows
results_df.head()

,Query,Product_Title,Product_Description,Image_Link,Similarity_Score
0,Replacement Filter Pack Compatible with Coway ...,AP-1216-FP Replacement Filter Compatible with ...,Brand\tByseacore Material\tPolypropylene Produ...,https://m.media-amazon.com/images/I/817zez3TPb...,0.566791
1,Replacement Filter Pack Compatible with Coway ...,PUREBURG AP-1216L Replacement True HEPA Filter...,Brand\tPUREBURG Material\tTrue HEPA MERV Ratin...,https://m.media-amazon.com/images/I/71cOAMPFfk...,0.540653
2,Replacement Filter Pack Compatible with Coway ...,AP-1216-FP Replacement Filter for Coway AP-121...,"Brand\tDAODEFUL Material\tTrue HEPA, Active Ca...",https://m.media-amazon.com/images/I/71yzYQwGUG...,0.565676
3,Replacement Filter Pack Compatible with Coway ...,1216L Replacement Filter Compatible with Coway...,Brand\tByseacore Material\tPolypropylene Produ...,https://m.media-amazon.com/images/I/81tFK-tnbU...,0.543476
4,Replacement Filter Pack Compatible with Coway ...,LifeSupplyUSA Complete Replacement Filter Set ...,Brand\tLifeSupplyUSA Material\tFiberglass Prod...,https://m.media-amazon.com/images/I/71qcUBK5xp...,0.615913


In [34]:
import altair as alt

# Group by query and calculate the average similarity score
average_similarity = results_df.groupby('Query')['Similarity_Score'].mean().reset_index()

# Create the Altair chart
chart = alt.Chart(average_similarity).mark_bar().encode(
    x='Query',
    y='Similarity_Score'
).properties(
    title='Average Similarity Scores for Each Query',
    width=600,
    height=400
)

chart


alt.Chart(...)

## Calculate the average similarity score across the dataset

In [35]:
# Calculate the mean of similarity scores
average_similarity = results_df["Similarity_Score"].mean()
print(f"Average Similarity Score: {average_similarity:.4f}")

Average Similarity Score: 0.5373


# Calculate the text and image Image similarity separately and then find the combined cosine similarity between query and recommended products.

In [36]:
def get_query_vs_recommendations_similarity(query, product_title, product_description, image_url):
    """
    Compute cosine similarity between:
    - The search query (text)
    - Each recommended product (text + image)

    Returns:
        dict: {"text_similarity": float, "image_similarity": float, "combined_similarity": float}
    """
    try:
        # Text Embeddings
        with torch.no_grad():
            query_text_embed = model.encode_text(clip.tokenize([query]).to(device))
            product_text = f"{product_title} | {product_description}"
            product_text_embed = model.encode_text(clip.tokenize([product_text], truncate=True).to(device))

            # Image Embedding
            response = requests.get(image_url, timeout=5)
            image = Image.open(BytesIO(response.content)).convert("RGB")
            image_embed = model.encode_image(preprocess(image).unsqueeze(0).to(device))

        # Convert to numpy arrays (detach first)
        query_np = query_text_embed.cpu().detach().numpy()
        product_text_np = product_text_embed.cpu().detach().numpy()
        image_np = image_embed.cpu().detach().numpy()

        # Compute Cosine Similarities
        text_sim = cosine_similarity(query_np, product_text_np)[0][0]
        image_sim = cosine_similarity(query_np, image_np)[0][0]

        # Combined similarity (average of text and image)
        combined_sim = (text_sim + image_sim) / 2

        return {
            "text_similarity": float(text_sim),
            "image_similarity": float(image_sim),
            "combined_similarity": float(combined_sim)
        }

    except Exception as e:
        print(f"Error processing {product_title}: {str(e)}")
        return {
            "text_similarity": 0.0,
            "image_similarity": 0.0,
            "combined_similarity": 0.0
        }

In [37]:
# Compute similarities for each query-product pair
results = []
for _, row in df_query.iterrows():
    sim_scores = get_query_vs_recommendations_similarity(
        query=row["Queries"],
        product_title=row["Product_title"],
        product_description=row["Product_description"],
        image_url=row["Image_link"]
    )
    results.append({
        "Query": row["Queries"],
        "Product_Title": row["Product_title"],
        "Text_Similarity": sim_scores["text_similarity"],
        "Image_Similarity": sim_scores["image_similarity"],
        "Combined_Similarity": sim_scores["combined_similarity"]
    })

# Convert to DataFrame
similarity_df = pd.DataFrame(results)

In [38]:
similarity_df.head()

,Query,Product_Title,Text_Similarity,Image_Similarity,Combined_Similarity
0,Replacement Filter Pack Compatible with Coway ...,AP-1216-FP Replacement Filter Compatible with ...,0.868634,0.324821,0.596727
1,Replacement Filter Pack Compatible with Coway ...,PUREBURG AP-1216L Replacement True HEPA Filter...,0.827838,0.319674,0.573756
2,Replacement Filter Pack Compatible with Coway ...,AP-1216-FP Replacement Filter for Coway AP-121...,0.846214,0.330337,0.588276
3,Replacement Filter Pack Compatible with Coway ...,1216L Replacement Filter Compatible with Coway...,0.868004,0.325085,0.596544
4,Replacement Filter Pack Compatible with Coway ...,LifeSupplyUSA Complete Replacement Filter Set ...,0.816391,0.335407,0.575899


In [39]:
# Calculate the mean of similarity scores
average_similarity = similarity_df["Combined_Similarity"].mean()
print(f"Average Similarity Score: {average_similarity:.4f}")

Average Similarity Score: 0.5067


## So, Better Average Similarity Score: 0.5373 (Multimodel Embedding Using CLIP Model)

